1- **Twitter EXERCISE:** 

Given the starting user "eloipuertas", retrieve the user name, the date of the accoount creation and the last tweet date and text for each follower. Store them to a database called 'twitter' and into a collection called 'info'. The attributes of the document are: 'screen_name', 'created_at', 'last_tweet_text' and 'last_tweet_created_at'. 


In [2]:
import json
import pymongo
import tweepy
import time

def get_last_user_tweet(user_id):
    status = api.user_timeline(user_id)
    return status

# Read twitter credentials
# with open('./twt_credentials.json') as f:
#     data = json.load(f)
    
with open('APIKeys.txt', 'r') as f:
    [consumer_key,consumer_secret,access_key, access_secret]=f.read().splitlines()

#--------------------------------------------------------------------------------
    
try:
  
    with open("credentials.txt", 'r') as f:
        [name,password,url]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
    print ("Connected successfully!!!")
    
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 

# Create the "twitter" database and the "info" collection
db = conn["twitter"]
collection = db.info

#Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


#Do something
USER_NAME = "Neoares92"
user = api.get_user(id=USER_NAME)

screen_name = user.screen_name
created_at = user._json["created_at"]

#Retrieving all the followers
followers = []
for page in tweepy.Cursor(api.followers_ids, screen_name=USER_NAME).pages():
    followers.extend(page)
    time.sleep(5)  #This should be 60 to avoid limit rates

print("Storing users data...")

for follower in followers:
    follower_user = api.get_user(id=follower)
    
    # Check if the user is protected to avoid errors
    if follower_user.protected:
        pass
    else:
        # Check if there is any tweet in the account and then we store the data
        status = get_last_user_tweet(follower)
        if status:
            collection.insert_one({
                "screen_name": follower_user.screen_name,
                "created_at": follower_user._json["created_at"],
                "last_tweet_text": status[0].text,
                "last_tweet_created_at": str(status[0].created_at)
            })
        else:
            pass
        
print("Data stored!")

Connected successfully!!!
Storing users data...
Data stored!


In [3]:
for x in collection.find({}, {"_id":0, "screen_name": 1, "created_at": 1, "last_tweet_text": 1, "last_tweet_created_at": 1}):  
    print(x)

{'screen_name': 'Clara_47687', 'created_at': 'Mon Jul 23 23:23:11 +0000 2012', 'last_tweet_text': 'La curva más bonita de una mujer es su sonrisa 😊 https://t.co/uMhUI0cMZZ', 'last_tweet_created_at': '2020-09-24 13:51:57'}
{'screen_name': 'miq_m', 'created_at': 'Wed May 21 00:14:29 +0000 2008', 'last_tweet_text': 'RT @martacava: On són les dones en un vídeo on es recomanen dones escriptores? https://t.co/RIMziMhG5F', 'last_tweet_created_at': '2020-10-19 14:01:42'}
{'screen_name': 'Goshita_', 'created_at': 'Wed Jun 28 13:21:00 +0000 2017', 'last_tweet_text': 'RT @ramonciuus: entrar a ikea y fantasear con como sería la casa de mis sueños me representa demasiado', 'last_tweet_created_at': '2020-10-20 09:39:47'}
{'screen_name': 'g_marrugat', 'created_at': 'Fri Jul 01 18:16:19 +0000 2016', 'last_tweet_text': 'RT @azti_brta: Hemos oído que, durante estos días, se han agotado ciertos productos en las estanterías de los supermercados. Pero ¿cómo ha…', 'last_tweet_created_at': '2020-05-12 08:17:

2- **Selenium EXERCISE:** 
<ul>
<li> Open a browser 

<li>Go to tripadvisor/Restaurants

<li>Find the search text box

<li>Clear it, input the query "Sant Cugat" and send it

<li>Go to "Restarurants" and get the name and link of the top 10 restaurants in Sant cugat
<li> Store them into a database called 'TripAdvisor' and a collection called 'Restaurants' with the documents atributes 'name' and 'link'
</ul>

</div>

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import pymongo

# Create the db
try:
  
    with open("credentials.txt", 'r') as f:
        [name,password,url]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
    print ("Connected successfully!!!")
    
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
db = conn['TripAdvisor']
collection = db.Restaurants

# CHANGE THE PATH WHEN YOU CORRECT IT
driver = webdriver.Chrome("chromedriver")
#---------------------------------------------------------------------------
driver.get('https://www.tripadvisor.es/Restaurants')

# find the search text box and write and look for Sant Cugat restaurants
text_box = driver.find_element_by_xpath('//*[@id="component_6"]/div/div/form/input[1]')
text_box.click()
text_box.send_keys('Sant Cugat del Vallès')
time.sleep(5)
driver.find_element_by_css_selector("a[href*='Sant_Cugat']").click()

# get top 10 restaurant
results = driver.find_element_by_xpath('//*[@id="component_2"]/div')
items = results.find_elements_by_css_selector('a[href*=REVIEWS]')
items2 = results.find_elements_by_class_name('wQjYiB7z')

names = []
links= []

for item, item2 in zip(items,items2):
    s = item2.get_attribute('innerHTML')
    restaurant_name = re.search("<!-- -->. <!-- -->(.*?)</a></span>", s)
    # filter promoted ones (Return None on search of pattern)
    if restaurant_name != None:
        names.append(restaurant_name.group(1))
        links.append(item.get_attribute('href').strip('#REVIEWS'))
        

names = names[:10]  
links = links[:10]

for name, link in zip(names, links):
    collection.insert_one({"restaurant": name, "link": link})

Connected successfully!!!


In [8]:
for x in collection.find({}, {"_id":0, "restaurant": 1, "link": 1}):  
    print(x)

{'restaurant': 'Piaceri D’ Italia Ristorante Pizzeria', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d19084243-Reviews-Piaceri_D_Italia_Ristorante_Pizzeria-Sant_Cugat_del_Valles_Catalonia.html'}
{'restaurant': 'Restaurant Brau', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d10195584-Reviews-Restaurant_Brau-Sant_Cugat_del_Valles_Catalonia.html'}
{'restaurant': 'La Rita', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d10365477-Reviews-La_Rita-Sant_Cugat_del_Valles_Catalonia.html'}
{'restaurant': 'Sabàtic', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d10167691-Reviews-Sabatic-Sant_Cugat_del_Valles_Catalonia.html'}
{'restaurant': 'Némesis Gastronomia', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d11892809-Reviews-Nemesis_Gastronomia-Sant_Cugat_del_Valles_Catalonia.html'}
{'restaurant': 'El vi de deu', 'link': 'https://www.tripadvisor.es/Restaurant_Review-g1080422-d7076969-Reviews-El_Vi_de_Deu-Sant_Cu